# Segmenting and Clustering Neighborhoods in Toronto

#### All the 3 tasks of web scraping, cleaning and clustering are implemented in the same notebook for the ease of evaluation.


## Part 1

Import libraries

In [1]:
import pandas as pd
import numpy as np

Download tables from wikipedia page as pandas dataframes

In [2]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(wiki_url, header=0)

Select the first (and only) dataframe and print the head

In [3]:
df=dfs[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Remove lines with Borough column not assigned

In [4]:
df=df[df['Borough']!='Not assigned']
#Alternative method
#df = df.drop(df[(df.Borough == "Not assigned")].index)

Join rows with same postal code and borough, by listing the neighborhoods and reset the indices

In [5]:
df = df.groupby(['Postal code','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)

Replace empty Neighborhood with corresponding Borough

In [6]:
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)
#Alternative method
#df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'], df['Neighborhood'])

Print the shape of the dataframe

In [7]:
df.shape

(103, 3)

## Part 2

Importing the csv file containing the latitudes and longitudes

In [8]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two dataframes in order to add Latitude and Longitude to the neighborhooods dataframe

In [9]:
coordinates.rename(columns={'Postal Code':'Postal code'},inplace=True)
df2 = pd.merge(df,coordinates,on='Postal code')
df2.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


## Part 3

Filtering in order to obtain only the Boroughs containing the word "Toronto"

In [10]:
df3 = df2[df2['Borough'].str.contains('Toronto',regex=False)]
df3.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


Installing the required libraries

In [11]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: ...working... done

# All requested packages already installed.



Visualizing all the Neighbourhoods of the above dataframe using Folium

In [12]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df3['Latitude'],df3['Longitude'],df3['Borough'],df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

#### In case the map doesn't appear on GitHub, look in the README for an image

Using KMeans for the clustering of the neighborhoods

In [13]:
k=5
toronto_clustering = df3.drop(['Postal code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df3.insert(0, 'Cluster Labels', kmeans.labels_)
df3.head()

,Cluster Labels,Postal code,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,0,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Visualizing all the clustered Neighbourhoods using Folium

In [14]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood'], df3['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### In case the map doesn't appear on GitHub, look in the README for an image